# Input Processing

In [7]:
data = open("data/19.txt", "r").read()

In [8]:
from dataclasses import dataclass

@dataclass
class Blueprint:
    id: int
    ore_robot_ore_cost: int
    clay_robot_ore_cost: int
    obsidian_robot_ore_cost: int
    obsidian_robot_clay_cost: int
    geode_robot_ore_cost: int
    geode_robot_obsidian_cost: int

In [9]:
import re

pattern = re.compile(r"Blueprint (\d+):\D+(\d+)\D+(\d+)\D+(\d+)\D+(\d+)\D+(\d+)\D+(\d+) \w+\.")

blueprints: list[Blueprint] = []
for match in pattern.findall(data):
    blueprints.append(Blueprint(*(int(group) for group in match)))

# Part 1

In [ ]:
def add_s(test_num: int):
    if test_num != 0:
        return "s"
    return ""

In [ ]:
def rm_s(test_num: int):
    if test_num != 0:
        return ""
    return "s"

In [ ]:
import enum
class PathMode(enum.Enum):
    TRY_MAKE = enum.auto()


In [10]:
import numpy as np

def process_blueprint(test_blueprint: Blueprint):

    # ore_robots, clay_robots, obsidian_robots, geode_robots,
    # ore, clay, obsidian, geode,
    # timer, debug_string
    stack: list[tuple[PathMode, np.ndarray, np.ndarray, np.ndarray, int, str]] = [(
        PathMode.TRY_MAKE, # mode
        np.array([]), # robots: ore, clay, obsidian, geode
        np.array([]), # inventory: ore, clay, obsidian, geode
        np.array([]), # future_robots: ore, clay, obsidian, geode
        0,
        ""
    )]
    max_geode = 0

    while len(stack) > 0:
        hand = stack.pop()
        mode, ore_robots, clay_robots, obsidian_robots, geode_robots, ore, clay, obsidian, geode, timer, debug_string = hand

        if timer > 24:
            max_geode = max(geode, max_geode)
            if geode == max_geode:
                max_complete = hand
            continue

        match mode:
            case PathMode.TRY_MAKE:


        debug_string += f"== Minute {timer} =="

        # Collect ore
        if ore_robots:
            ore += ore_robots
            debug_string += f"{ore_robots} ore-collecting robot{add_s(ore_robots)} collect{rm_s(ore_robots)} {ore_robots} ore; you now have {ore} ore."
        if clay_robots:
            clay += clay_robots
            debug_string += f"{clay_robots} clay-collecting robot{add_s(clay_robots)} collect{rm_s(clay_robots)} {clay_robots} clay; you now have {clay} clay."
        if obsidian_robots:
            obsidian += obsidian_robots
            debug_string += f"{obsidian_robots} obsidian-collecting robot{add_s(obsidian_robots)} collect{rm_s(obsidian_robots)} {obsidian_robots} obsidian; you now have {obsidian} obsidian."
        if geode_robots:
            geode += geode_robots
            debug_string += f"{geode_robots} geode-collecting robot{add_s(geode_robots)} collect{rm_s(geode_robots)} {geode_robots} ore; you now have {geode} open geode{add_s(geode)}."
    return max_complete

[Blueprint(id=1, ore_robot_ore_cost=4, clay_robot_ore_cost=2, obsidian_robot_ore_cost=3, obsidian_robot_clay_cost=14, geode_robot_ore_cost=2, geode_robot_obsidian_cost=7),
 Blueprint(id=2, ore_robot_ore_cost=2, clay_robot_ore_cost=3, obsidian_robot_ore_cost=3, obsidian_robot_clay_cost=8, geode_robot_ore_cost=3, geode_robot_obsidian_cost=12)]